In [1]:
%scala
import org.apache.spark.sql.streaming._

spark.streams.addListener(new StreamingQueryListener() {
  override def onQueryStarted(started: StreamingQueryListener.QueryStartedEvent): Unit = {
      println("Started: " + started.id)
  }
  override def onQueryTerminated(terminated: StreamingQueryListener.QueryTerminatedEvent): Unit = {
      println("Terminated: " + terminated.id)
  }
  override def onQueryProgress(progress: StreamingQueryListener.QueryProgressEvent): Unit = {
      println("Progress: " + progress.progress)
  }
})

In [2]:
%scala
import org.apache.spark.eventhubs._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

// Build connection string with the above information
val connectionString = ConnectionStringBuilder("<Event hub connection string>")
  .setEventHubName("<Event hub name>")
  .build

val customEventhubParameters =
  EventHubsConf(connectionString)
  .setMaxEventsPerTrigger(5)

val df = spark.readStream.format("eventhubs").options(customEventhubParameters.toMap).load()

df.writeStream.outputMode("append").format("console").option("truncate", false).start().awaitTermination()